In [1]:
from gezi import tqdm
import melt
import gezi
logging = gezi.logging
import tensorflow as tf
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', None, '')

tensorflow_version: 1.14.0
torch_version: 1.3.1+cu100
Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
feature_index = '/search/odin/publicData/CloudS/rank/feature_indexes/tuwen/16/2020012022/feature_index_field.utf8'

In [4]:
m = {}
m2 = {}
for line in tqdm(open(feature_index)):
  index_name, index_id, field_id = line.strip().split('\t')
  m[int(index_id)] = index_name
  m2[int(field_id)] = index_name.split('\a')[0]

15327887it [01:30, 169652.28it/s]


In [5]:
from projects.feed.rank.src.tfrecord_dataset import Dataset

In [6]:
import sys
FLAGS(sys.argv)

['/home/gezi/env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py']

In [7]:
tf.enable_eager_execution()

In [8]:
dataset = Dataset('valid')

In [9]:
files = gezi.list_files('/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v1/tfrecords/2020012022')
total = melt.get_num_records(files) 

get_num_records:   0%|          | 0/54 [00:00<?, ?it/s]

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


get_num_records: 100%|##########| 54/54 [00:00<00:00, 913.87it/s]


In [10]:
batch_size = 1
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)

In [11]:
import numpy as np
bad_fields = set()
bad_count = 0
for i, (x, _) in tqdm(enumerate(batches), total=num_steps, ascii=True, desc='loop'):
  for key in x.keys():
    x[key] = x[key].numpy()
  indexes = x['index'][0]
  fields = x['field'][0]
  values = x['value'][0]
  mid = x['mid'][0].decode()
  docid = x['docid'][0].decode()
  flags = values == 0
  zero_val_indexes = indexes[flags]
  zero_val_fields = set(fields[flags])
  bad_fields.update(zero_val_fields)
  if len(bad_fields) > bad_count:
    bad_count = len(bad_fields)
    print('----------------------')
    print(i, bad_count)
    for x in bad_fields:
      print(m2[x])
#   fields2 = np.asarray([fid in zero_val_fields for fid in fields])
#   indexes2 = indexes[fields2]
#   names = [m.get(index, None) for index in zero_val_indexes]
#   names2 = [m.get(index, None) for index in indexes2]
#   vals2 = values[fields2]
#   if names:
#     print(i, mid, docid)
#     for name, val in zip(names2, vals2):
#       print(name, val)
#   if i == 100:
#     break
print(len(bad_fields))
print(bad_fields)

loop:   0%|          | 87/1945218 [00:00<1:33:43, 345.89it/s]

----------------------
0 1
IATKW
----------------------
4 2
IATTP
IATKW
----------------------
18 9
MITATTP
IATTP
IATKW
MITATWCNTTP
MITATQUALSTP
MITATPCNTTP
MITATPORNSTP
CITATCOTP
MITATDURTP
----------------------
20 10
MITATTP
IATTP
IATKW
MITATWCNTTP
MITATQUALSTP
IATKWSE
MITATPCNTTP
MITATPORNSTP
CITATCOTP
MITATDURTP
----------------------
42 16
MITATTP
IATTP
IATKW
MITATQUALSKW
MITATWCNTTP
MITATQUALSTP
IATKWSE
MITATDURKW
MITATPORNSKW
MITATPCNTTP
MITATKW
MITATPCNTKW
MITATWCNTKW
MITATPORNSTP
CITATCOTP
MITATDURTP


loop:  10%|9         | 189635/1945218 [05:29<45:46, 639.30it/s]  

----------------------
189555 22
MITATTP
MITATQUALSTG
IATTP
IATKW
MITATWCNTTP
MITATQUALSTP
IATKWSE
MITATDURKW
MITATTG
MITATPCNTTP
MITATPCNTKW
MITATWCNTKW
MITATQUALSKW
MITATPORNSTP
MITATWCNTTG
CITATCOTP
MITATPORNSTG
MITATPCNTTG
MITATDURTG
MITATPORNSKW
MITATKW
MITATDURTP


loop: 100%|##########| 1945218/1945218 [55:16<00:00, 586.50it/s] 

22
{-5088830181173556222, 5990307752623814594, -1119262723919422650, -7697086540126162871, 2400462395024243787, -2519220192420063411, 2068597903886099341, 2344283115461994379, 5573857014160680586, -2669211736745653421, 8650559348721361363, 5228992160750648825, 8593059423197761109, -2729318733414106023, -1791070171177239975, -5817666613973072483, -8311434859857709405, -378668300094641051, -7420814268695332945, -4919644448854171470, -7650487256300491330, -5667364694360982464}
